In [26]:
import pandas as pd
import numpy as np

# Escaladores
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Train, Test
from sklearn.model_selection import train_test_split

# Modelos

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor



# Métricas
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error

In [28]:
df_encoded = pd.read_csv('data/df_depuracion.csv')

## TRAIN TEST

In [4]:
X = df_encoded.drop("prices_per_night", axis = 1)
y = df_encoded["prices_per_night"]
print(X.shape, y.shape)

(2497, 25) (2497,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

X_train: (1997, 25), y_train: (1997,)
X_test: (500, 25), y_test: (500,)


## Escalado

In [7]:
x_scaler = MinMaxScaler()
X_train = x_scaler.fit_transform(X_train)
X_test = x_scaler.transform(X_test)

y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(np.array(y_train).reshape(-1, 1))
y_test = y_scaler.transform(np.array(y_test).reshape(-1, 1))

### Buscar el mejor modelo con r2 mas alto 

In [9]:
# Definimos los modelos
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Support Vector Regressor": SVR(),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "LightGBM": LGBMRegressor(random_state=42),
    "MLP Regressor": MLPRegressor(random_state=42)
}

In [10]:
# Lista para almacenar los resultados
resultados_lista = []

# Bucle para entrenar cada modelo y calcular métricas
for model_name, model in models.items():
    model.fit(X_train, y_train.ravel())
    
    y_hat = model.predict(X_test)
    
    # Desescalado de las predicciones
    y_test_inv = y_scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()
    y_hat_inv = y_scaler.inverse_transform(y_hat.reshape(-1, 1)).ravel()
    
    # Cálculo de métricas
    mae = mean_absolute_error(y_test_inv, y_hat_inv)
    mse = mean_squared_error(y_test_inv, y_hat_inv)
    rmse = root_mean_squared_error(y_test_inv, y_hat_inv)
    r2 = r2_score(y_test_inv, y_hat_inv)
    
    # Almacenar resultados
    resultados_lista.append({
        "model_name": model_name,
        "mae": mae,
        "mse": mse,
        "rmse" : rmse,
        "r2_score": r2
    })

# Crear DataFrame y ordenar por r2_score
resultados = pd.DataFrame(resultados_lista)
resultados = resultados.sort_values(by="r2_score", ascending=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000099 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 629
[LightGBM] [Info] Number of data points in the train set: 1997, number of used features: 24
[LightGBM] [Info] Start training from score 0.143845


In [11]:
resultados

,model_name,mae,mse,rmse,r2_score
5,LightGBM,9.460706,226.801294,15.059923,0.870380
4,XGBoost,7.861663,245.519606,15.669065,0.859682
1,Random Forest,8.741529,266.327709,16.319550,0.847790
3,Gradient Boosting,13.894323,375.717703,19.383439,0.785272
0,Linear Regression,18.315253,644.651826,25.389995,0.631572
6,MLP Regressor,19.612846,684.445896,26.161917,0.608829
2,Support Vector Regressor,21.229867,703.222111,26.518335,0.598098
